In [2]:
import random
import numpy as np

#Converts an order in the form of a string of numbers into a matrix describing which players are ranked above which other players. 
def orderMatrix(order, numPlayers):
    matrix = np.zeros((numPlayers,numPlayers))
    unrankedPlayers = list(range(1, numPlayers+1))
    for x in order:
        if x != "0":
            unrankedPlayers.remove(int(x))
    row = np.ones(numPlayers)
    for x in order:
        #The row starts as all 1s, indicating a preference above all other players. As each player is then placed, the 1 in their position is removed
        #to indicate that subsequently ranked players aren't preferred over formerly ranked players.
        if x != "0":
            row[int(x)-1] -= 1
            matrix[int(x)-1] += row
        else:
            #A '0' in the ranking is a placeholder for "all other players in an unspecified order", this system is to speed up difficult and
            #inconsequential decision making.
            for y in unrankedPlayers:
                row[y-1] -= 1
            for y in unrankedPlayers:
                matrix[y-1] += row
    return matrix;

#Takes the score matrix and for each combination of two players counts how many times they were ranked above one another. Whichever player was ranked
#higher more often is given a 'win', and the overall order is decided by how many 'wins' each player has. Players with the same number of wins tie.
def rankings(score, numPlayers):
    wins = np.zeros(numPlayers)
    for x in range(numPlayers):
        for y in range(numPlayers):
            if x < y:
                if score[x, y] > score [y, x]:
                    wins[x] += 1
                elif score[y, x] > score [x, y]:
                    wins[y] += 1
    ranks = np.zeros((3, numPlayers))
    ranks[0] += np.arange(1, numPlayers+1)
    ranks[1] += wins
    for x in range(numPlayers):
        playersAbove = 0
        for y in range(numPlayers):
            if ranks[1, x] < ranks[1, y]:
                playersAbove += 1
        ranks[2,x] = playersAbove + 1
    return ranks;
#The 'ranks' matrix is unintuitive and probably needs a better name. The top row is just the player numbers, which is normally handled by indices but
#this matrix gets sorted later so this is necessary to keep track of the players. The second row is how many wins each player has and the third is their
#position in the overall ranking.

#This just makes sure that the strings entered by players are valid orders. A valid order is a string of numbers less than or equal to the number of
#players and with no repeats.
def isValid(order, numPlayers):
    try:
        values = []
        for x in order:
            if int(x) <= numPlayers:
                values.append(int(x))
            else:
                return False;
        if len(values) == len(list(set(values))):
            return True;
        else:
            return False;
    except:
        return False;

players = ["Matt", "Rachel", "Sam", "Sarah", "Tom"]
numPlayers = len(players)

categories = ["Peg-ability"]

print("Player numbers are as follows:")
for x in range(numPlayers):
    print(x+1, players[x])

Player numbers are as follows:
1 Matt
2 Rachel
3 Sam
4 Sarah
5 Tom


In [4]:
score = np.zeros((numPlayers,numPlayers))
category = random.choice(categories)
print("The category is:",category)

for p in range(5):    
    validOrder = False
    while not(validOrder):
        order = str(input("Enter your chosen order as a string of numbers."))
        if isValid(order, numPlayers):
            validOrder = True
        else:
            print("The order you entered is not valid.")
    score += orderMatrix(order, numPlayers)

print(score)

Enter your chosen order as a string of numbers. hello


The order you entered is not valid.


Enter your chosen order as a string of numbers. 3141592


The order you entered is not valid.


Enter your chosen order as a string of numbers. 11111


...

The order you entered is not valid.


Enter your chosen order as a string of numbers. 2.4


The order you entered is not valid.


Enter your chosen order as a string of numbers. 123
Enter your chosen order as a string of numbers. 1204
Enter your chosen order as a string of numbers. 21534
Enter your chosen order as a string of numbers. 215
Enter your chosen order as a string of numbers. 1325


[[0. 3. 5. 5. 5.]
 [2. 0. 4. 5. 5.]
 [0. 1. 0. 4. 2.]
 [0. 0. 0. 0. 0.]
 [0. 0. 2. 4. 0.]]


In [5]:
results = rankings(score, numPlayers)
#No prizes for guessing which is the only line in this whole thing that I basically ripped from Stack Overflow
sortedResults = results[:,results[2, :].argsort()]
print(results,"\n", sortedResults)

[[1. 2. 3. 4. 5.]
 [4. 3. 1. 0. 1.]
 [1. 2. 3. 5. 3.]] 
 [[1. 2. 3. 5. 4.]
 [4. 3. 1. 1. 0.]
 [1. 2. 3. 3. 5.]]


In [53]:
#2 points for a guess being exact, 1 point for being one away. I haven't written the bit for entering guesses yet. I was going to do a more complicated
#system for determining the number of points to give with tied rankings but that's a pretty low priority, I can do that once the rest of the game
#actually works.
guesses = np.zeros((3, 5))
guesses[0] += np.array([1, 1, 5, 1, 4])
guesses[1] += results[2]
for x in range(5):
    if guesses[0,x] == guesses[1,x]:
        guesses[2,x] = 2
    elif int(abs(guesses[0,x]-guesses[1,x])) == 1:
        guesses[2,x] = 1
print(guesses)

[[1. 1. 5. 1. 4.]
 [2. 1. 4. 3. 4.]
 [1. 2. 1. 0. 2.]]


In [ ]:
#This is something ChatGPT spat out when I asked it how to make a discord bot, no idea if it works yet or what all of it does.
import discord
from discord.ext import commands

# Bot token
TOKEN = 'your_bot_token_here'

# Command prefix for direct messages
bot = commands.Bot(command_prefix='!', intents=discord.Intents.default())


@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name}')


@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    if isinstance(message.channel, discord.DMChannel):
        # Concatenate inputs from multiple players
        inputs = message.content

        # Get the guild (server) you want to send the output message to
        guild = bot.get_guild(your_guild_id_here)  # Replace your_guild_id_here with the guild ID

        # Get the channel within the guild where you want to send the output message
        output_channel = guild.get_channel(your_channel_id_here)  # Replace your_channel_id_here with the channel ID

        # Send the concatenated inputs as a message to the specified channel
        await output_channel.send(f'Concatenated inputs: {inputs}')

    await bot.process_commands(message)


bot.run(TOKEN)
